In [5]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency
import numpy as np
import scipy.stats as st
import pandas as pd
from scipy.stats import f_oneway
import re

### 6개월치 데이터 합쳐서 저장.

In [8]:
# 1) 마케팅정보 파일들이 모여 있는 폴더
DATA_DIR = Path("open/train/6.채널정보")

# 2) 201807~201812 사이의 train-마케팅정보 Parquet 파일만 모으기
parquet_files = sorted(
    DATA_DIR.glob("2018??_train_*.parquet"),          # 201807, 201808 …
    key=lambda p: re.match(r"(\d{6})", p.name).group(1)
)

print("읽을 파일:", [p.name for p in parquet_files])

# 3) 파일별로 읽어 month 컬럼 붙이고 리스트에 저장
df_list = []
for fp in parquet_files:
    df = pd.read_parquet(fp)
    df["month"] = fp.stem[:6]       # '201807' 같은 연월
    df_list.append(df)

# 4) 하나로 병합
df_marketing = pd.concat(df_list, ignore_index=True)

# 5) (선택) 새 파일로 저장
output_path = DATA_DIR / "train_채널정보_201807_201812.parquet"
df_marketing.to_parquet(output_path, index=False)

# 6) 확인
print("병합 완료! 행·열:", df_marketing.shape)
display(df_marketing.head())
display(df_marketing.tail())

읽을 파일: ['201807_train_채널정보.parquet', '201808_train_채널정보.parquet', '201809_train_채널정보.parquet', '201810_train_채널정보.parquet', '201811_train_채널정보.parquet', '201812_train_채널정보.parquet']
병합 완료! 행·열: (2400000, 106)


,기준년월,ID,인입횟수_ARS_R6M,이용메뉴건수_ARS_R6M,인입일수_ARS_R6M,인입월수_ARS_R6M,인입후경과월_ARS,인입횟수_ARS_B0M,이용메뉴건수_ARS_B0M,인입일수_ARS_B0M,...,당사PAY_방문월수_R6M,당사멤버쉽_방문횟수_B0M,당사멤버쉽_방문횟수_R6M,당사멤버쉽_방문월수_R6M,OS구분코드,홈페이지_금융건수_R6M,홈페이지_선결제건수_R6M,홈페이지_금융건수_R3M,홈페이지_선결제건수_R3M,month
0,201807,TRAIN_000000,10회 이상,10회 이상,8,6,0,2,6,2,...,0,22,221,6,Android,0,0,0,0,201807
1,201807,TRAIN_000001,1회 이상,1회 이상,0,0,0,0,0,0,...,0,0,0,0,None,0,0,0,0,201807
2,201807,TRAIN_000002,1회 이상,1회 이상,1,1,0,2,5,1,...,0,0,0,0,Android,11,6,5,5,201807
3,201807,TRAIN_000003,10회 이상,10회 이상,10,6,0,2,6,2,...,0,23,219,6,Android,0,0,0,0,201807
4,201807,TRAIN_000004,1회 이상,1회 이상,0,0,0,0,0,0,...,0,0,0,0,Android,0,0,0,0,201807


,기준년월,ID,인입횟수_ARS_R6M,이용메뉴건수_ARS_R6M,인입일수_ARS_R6M,인입월수_ARS_R6M,인입후경과월_ARS,인입횟수_ARS_B0M,이용메뉴건수_ARS_B0M,인입일수_ARS_B0M,...,당사PAY_방문월수_R6M,당사멤버쉽_방문횟수_B0M,당사멤버쉽_방문횟수_R6M,당사멤버쉽_방문월수_R6M,OS구분코드,홈페이지_금융건수_R6M,홈페이지_선결제건수_R6M,홈페이지_금융건수_R3M,홈페이지_선결제건수_R3M,month
2399995,201812,TRAIN_399995,1회 이상,1회 이상,0,0,0,0,0,0,...,0,0,0,0,None,0,0,0,0,201812
2399996,201812,TRAIN_399996,1회 이상,1회 이상,0,0,0,0,0,0,...,0,0,0,1,None,0,0,0,0,201812
2399997,201812,TRAIN_399997,1회 이상,1회 이상,0,0,0,0,0,0,...,0,0,0,0,None,0,0,0,0,201812
2399998,201812,TRAIN_399998,1회 이상,1회 이상,0,0,0,0,0,0,...,0,0,0,0,None,0,0,0,0,201812
2399999,201812,TRAIN_399999,1회 이상,1회 이상,0,0,0,0,0,0,...,0,0,0,0,None,0,0,0,0,201812


### 합친 데이터 불러와서 확인

In [11]:
df1 = pd.read_parquet('open/train/6.채널정보/파생/train_채널정보_201807_201812.parquet')
df1

,기준년월,ID,인입횟수_ARS_R6M,이용메뉴건수_ARS_R6M,인입일수_ARS_R6M,인입월수_ARS_R6M,인입후경과월_ARS,인입횟수_ARS_B0M,이용메뉴건수_ARS_B0M,인입일수_ARS_B0M,...,당사PAY_방문월수_R6M,당사멤버쉽_방문횟수_B0M,당사멤버쉽_방문횟수_R6M,당사멤버쉽_방문월수_R6M,OS구분코드,홈페이지_금융건수_R6M,홈페이지_선결제건수_R6M,홈페이지_금융건수_R3M,홈페이지_선결제건수_R3M,month
0,201807,TRAIN_000000,10회 이상,10회 이상,8,6,0,2,6,2,...,0,22,221,6,Android,0,0,0,0,201807
1,201807,TRAIN_000001,1회 이상,1회 이상,0,0,0,0,0,0,...,0,0,0,0,None,0,0,0,0,201807
2,201807,TRAIN_000002,1회 이상,1회 이상,1,1,0,2,5,1,...,0,0,0,0,Android,11,6,5,5,201807
3,201807,TRAIN_000003,10회 이상,10회 이상,10,6,0,2,6,2,...,0,23,219,6,Android,0,0,0,0,201807
4,201807,TRAIN_000004,1회 이상,1회 이상,0,0,0,0,0,0,...,0,0,0,0,Android,0,0,0,0,201807
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,201812,TRAIN_399995,1회 이상,1회 이상,0,0,0,0,0,0,...,0,0,0,0,None,0,0,0,0,201812
2399996,201812,TRAIN_399996,1회 이상,1회 이상,0,0,0,0,0,0,...,0,0,0,1,None,0,0,0,0,201812
2399997,201812,TRAIN_399997,1회 이상,1회 이상,0,0,0,0,0,0,...,0,0,0,0,None,0,0,0,0,201812
2399998,201812,TRAIN_399998,1회 이상,1회 이상,0,0,0,0,0,0,...,0,0,0,0,None,0,0,0,0,201812


In [45]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400000 entries, 0 to 2399999
Columns: 106 entries, 기준년월 to month
dtypes: int64(98), object(8)
memory usage: 1.9+ GB


In [13]:
df1.describe()

,기준년월,인입일수_ARS_R6M,인입월수_ARS_R6M,인입후경과월_ARS,인입횟수_ARS_B0M,이용메뉴건수_ARS_B0M,인입일수_ARS_B0M,방문월수_PC_R6M,방문후경과월_PC_R6M,방문일수_앱_R6M,...,당사PAY_방문횟수_B0M,당사PAY_방문횟수_R6M,당사PAY_방문월수_R6M,당사멤버쉽_방문횟수_B0M,당사멤버쉽_방문횟수_R6M,당사멤버쉽_방문월수_R6M,홈페이지_금융건수_R6M,홈페이지_선결제건수_R6M,홈페이지_금융건수_R3M,홈페이지_선결제건수_R3M
count,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,...,2400000.0,2400000.0,2400000.0,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06
mean,2.018095e+05,8.690288e-01,5.484083e-01,7.018121e-01,1.276996e-01,3.196038e-01,1.044304e-01,4.768792e-01,5.397019e+00,3.879475e+00,...,0.0,0.0,0.0,1.396040e+00,1.140625e+01,5.885179e-01,7.375242e-01,9.763875e-01,4.067225e-01,5.264267e-01
std,1.707825e+00,1.946545e+00,1.190408e+00,1.483387e+00,5.015725e-01,1.245971e+00,4.229458e-01,1.507584e+00,1.696640e+00,1.004851e+01,...,0.0,0.0,0.0,6.566001e+00,4.581328e+01,1.692597e+00,2.114160e+00,3.130675e+00,1.187245e+00,1.700495e+00
min,2.018070e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.018080e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.000000e+00,0.000000e+00,...,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.018095e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.000000e+00,0.000000e+00,...,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,2.018110e+05,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.000000e+00,0.000000e+00,...,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,2.018120e+05,1.200000e+01,6.000000e+00,5.000000e+00,3.000000e+00,7.000000e+00,2.000000e+00,6.000000e+00,6.000000e+00,6.400000e+01,...,0.0,0.0,0.0,6.300000e+01,2.590000e+02,6.000000e+00,1.500000e+01,3.000000e+01,8.000000e+00,1.600000e+01


In [17]:
# 중복된 행, 열 확인
df1.duplicated().sum()

0

### 전체 parquet 파일에 segment 컬럼 병합

In [21]:
# ── 1) 파일 읽기 
path_member   = Path("open/train/1.회원정보/201807_train_회원정보.parquet")
path_channel = Path("open/train/6.채널정보/파생/train_채널정보_201807_201812.parquet")

member_df   = pd.read_parquet(path_member)
channel_df = pd.read_parquet(path_channel)

# ── 2) Segment 컬럼만 추출 → ID 기준으로 병합
segment_df = member_df[["ID", "Segment"]]          # (필요 컬럼만)
channel_df = channel_df.merge(segment_df, on="ID", how="left")

# ── 4) 결과 저장 
# channel_df.to_parquet("open/train/6.채널정보/파생/train_채널정보_201807_201812_segment.parquet",
                    #index=False)

print("✔ Segment 컬럼 이동 완료!")

✔ Segment 컬럼 이동 완료!


In [23]:
df2 = pd.read_parquet('open/train/6.채널정보/파생/train_채널정보_201807_201812_segment.parquet')
df2

,기준년월,ID,인입횟수_ARS_R6M,이용메뉴건수_ARS_R6M,인입일수_ARS_R6M,인입월수_ARS_R6M,인입후경과월_ARS,인입횟수_ARS_B0M,이용메뉴건수_ARS_B0M,인입일수_ARS_B0M,...,당사멤버쉽_방문횟수_B0M,당사멤버쉽_방문횟수_R6M,당사멤버쉽_방문월수_R6M,OS구분코드,홈페이지_금융건수_R6M,홈페이지_선결제건수_R6M,홈페이지_금융건수_R3M,홈페이지_선결제건수_R3M,month,Segment
0,201807,TRAIN_000000,10회 이상,10회 이상,8,6,0,2,6,2,...,22,221,6,Android,0,0,0,0,201807,D
1,201807,TRAIN_000001,1회 이상,1회 이상,0,0,0,0,0,0,...,0,0,0,None,0,0,0,0,201807,E
2,201807,TRAIN_000002,1회 이상,1회 이상,1,1,0,2,5,1,...,0,0,0,Android,11,6,5,5,201807,C
3,201807,TRAIN_000003,10회 이상,10회 이상,10,6,0,2,6,2,...,23,219,6,Android,0,0,0,0,201807,D
4,201807,TRAIN_000004,1회 이상,1회 이상,0,0,0,0,0,0,...,0,0,0,Android,0,0,0,0,201807,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,201812,TRAIN_399995,1회 이상,1회 이상,0,0,0,0,0,0,...,0,0,0,None,0,0,0,0,201812,E
2399996,201812,TRAIN_399996,1회 이상,1회 이상,0,0,0,0,0,0,...,0,0,1,None,0,0,0,0,201812,D
2399997,201812,TRAIN_399997,1회 이상,1회 이상,0,0,0,0,0,0,...,0,0,0,None,0,0,0,0,201812,C
2399998,201812,TRAIN_399998,1회 이상,1회 이상,0,0,0,0,0,0,...,0,0,0,None,0,0,0,0,201812,E


In [25]:
df2['Segment'].isna().sum()

0

### 결측치 개수와 비율을 확인해본다.

In [27]:
# 1) 컬럼별 결측치 개수
na_cnt = df2.isna().sum()

# 2) 컬럼별 결측치 비율 (%)
na_pct = (df2.isna().mean() * 100).round(2)

# 3) 보기 좋게 합친 테이블
na_summary = pd.DataFrame({"missing_count": na_cnt,
                           "missing_pct": na_pct}) \
                .sort_values("missing_pct", ascending=False)

display(na_summary.head(20))      # 결측률 높은 상위 20개만 미리보기

,missing_count,missing_pct
OS구분코드,1633566,68.07
기준년월,0,0.00
상담건수_R6M,0,0.00
IB문의건수_APP_R6M,0,0.00
IB문의건수_SMS_R6M,0,0.00
IB문의건수_비밀번호_R6M,0,0.00
IB문의건수_명세서_R6M,0,0.00
IB문의건수_결제일변경_R6M,0,0.00
IB문의건수_정보변경_R6M,0,0.00
IB문의건수_할부_R6M,0,0.00


- OS구분코드 컬럼 외에는 결측치가 없다.
- OS구분코드 컬럼은 결측치의 비율이 68%나 되므로 삭제한다.

### 삭제할 컬럼들 정리
- 기존에 컬럼분석을 통해 제거하기로 한 컬럼들은 제거한다.
- OS구분코드 컬럼은 결측치가 너무 많아 제거한다.
- Segment를 합치면서 생성된 month 컬럼을 제거한다.
- 컬럼 내의 모든 데이터가 같은 컬럼은 제거한다.

### 제거1. 컬럼의 모든 데이터가 같은 컬럼

- 모든 데이터가 같은 컬럼 확인

In [29]:
constant_cols = [col for col in df2.columns
                 if df2[col].nunique(dropna=False) <= 1]

constant_cols

['인입횟수_금융_IB_R6M',
 '인입불만횟수_IB_R6M',
 '인입불만일수_IB_R6M',
 '인입불만월수_IB_R6M',
 '인입불만후경과월_IB_R6M',
 '인입불만횟수_IB_B0M',
 '인입불만일수_IB_B0M',
 'IB문의건수_한도_B0M',
 'IB문의건수_결제_B0M',
 'IB문의건수_할부_B0M',
 'IB문의건수_정보변경_B0M',
 'IB문의건수_결제일변경_B0M',
 'IB문의건수_명세서_B0M',
 'IB문의건수_비밀번호_B0M',
 'IB문의건수_SMS_B0M',
 'IB문의건수_APP_B0M',
 'IB문의건수_부대서비스_B0M',
 'IB문의건수_포인트_B0M',
 'IB문의건수_BL_B0M',
 'IB문의건수_분실도난_B0M',
 'IB문의건수_CA_B0M',
 'IB상담건수_VOC_B0M',
 'IB상담건수_VOC민원_B0M',
 'IB상담건수_VOC불만_B0M',
 'IB상담건수_금감원_B0M',
 'IB문의건수_명세서_R6M',
 'IB문의건수_APP_R6M',
 'IB상담건수_VOC_R6M',
 'IB상담건수_VOC민원_R6M',
 'IB상담건수_VOC불만_R6M',
 'IB상담건수_금감원_R6M',
 '불만제기건수_B0M',
 '불만제기건수_R12M',
 '당사PAY_방문횟수_B0M',
 '당사PAY_방문횟수_R6M',
 '당사PAY_방문월수_R6M']

- 모든 데이터가 같은 컬럼 제거

In [31]:
const_cols = df2.columns[df2.nunique(dropna=False) == 1]
#print("상수 컬럼:", list(const_cols))

# 3) 상수 컬럼 삭제
df3 = df2.drop(columns=const_cols)

for col in df3.columns:
    print(f"\n===== [ {col} ] =====")
    
    # 1) 값 분포 (결측치 포함)
    print("▶ value_counts (dropna=False)")
    print(df3[col].value_counts(dropna=False))
# 4) 결과 저장
#df3.to_parquet("201807_201812_segment_컬럼제거", index=False)
#print("상수 컬럼 제거 후 저장 완료!")


===== [ 기준년월 ] =====
▶ value_counts (dropna=False)
기준년월
201807    400000
201808    400000
201809    400000
201810    400000
201811    400000
201812    400000
Name: count, dtype: int64

===== [ ID ] =====
▶ value_counts (dropna=False)
ID
TRAIN_000000    6
TRAIN_266650    6
TRAIN_266672    6
TRAIN_266671    6
TRAIN_266670    6
               ..
TRAIN_133331    6
TRAIN_133330    6
TRAIN_133329    6
TRAIN_133328    6
TRAIN_399999    6
Name: count, Length: 400000, dtype: int64

===== [ 인입횟수_ARS_R6M ] =====
▶ value_counts (dropna=False)
인입횟수_ARS_R6M
1회 이상     2328933
10회 이상      71067
Name: count, dtype: int64

===== [ 이용메뉴건수_ARS_R6M ] =====
▶ value_counts (dropna=False)
이용메뉴건수_ARS_R6M
1회 이상     2313787
10회 이상      70027
30회 이상      10172
20회 이상       6014
Name: count, dtype: int64

===== [ 인입일수_ARS_R6M ] =====
▶ value_counts (dropna=False)
인입일수_ARS_R6M
0     1683964
1      266478
2      200257
3       74634
4       39222
5       26364
6       21823
8       20994
9       19334
7       19244

In [33]:
num_cols = len(df3.columns)
num_cols

71

### 제거2. 컬럼 분석을 통해 제거하고자 하는 컬럼들 + os구분코드 컬럼 + month 컬럼

In [35]:
cols_to_drop = [
    "인입후경과월_ARS", "인입후경과월_IB_R6M", "방문후경과월_PC_R6M", "방문후경과월_앱_R6M",
    "방문후경과월_모바일웹_R6M", "인입월수_ARS_R6M", "방문월수_PC_R6M",
    "방문월수_앱_R6M", "방문월수_모바일웹_R6M", "인입월수_IB_R6M",
    "당사멤버쉽_방문월수_R6M", "OS구분코드", "month"
]

df3 = df3.drop(columns=cols_to_drop)

In [37]:
num_cols = len(df3.columns)
num_cols

58

### 저장한다.

In [40]:
df3.to_parquet('open/train/6.채널정보/train_채널정보_전처리', index=False)